## MLP Classification by training on trajectory

<br> This notebook is adopted from the work: https://pubs.acs.org/doi/10.1021/acs.jpclett.1c01494
<br> Git repo: https://github.com/zzhang624/ML-SARS-CoV-2

In [1]:
#import modules
import os 
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [116]:
crRNA = np.load("../New_model_2/crRNA_dist_feature.npy")[:15000]
tgRNA = np.load("../New_model_2/tgRNA_dist_feature.npy")[:15000]

# crRNA = np.concatenate((crRNA_f1, crRNA_f2, crRNA_f3, crRNA_f4,crRNA_f5), axis=1)
print(crRNA.shape)
print(np.where(crRNA[:,:] == 0))

print(tgRNA.shape)
print(np.where(tgRNA[:,:] == 0))

# model5_training = np.load("model-5-HEPN1I-HEPN2/model-5-final-training-data.npy")
# print(model5_training.shape)
# print(model5_training)

(15000, 662)
(array([], dtype=int64), array([], dtype=int64))
(15000, 662)
(array([], dtype=int64), array([], dtype=int64))


In [122]:
df=pd.DataFrame(np.concatenate((crRNA,tgRNA),axis=0))

# print(df)

In [123]:
#inverse distance
df=1./df

# #add label
df['state'] = 'state'
df.iloc[:15000,-1] = 1
df.iloc[15000:,-1] = 2

#scaled/normalized by each feature
df_scaled = df.copy()
X = df_scaled.iloc[:,:-1]
X = (X-X.mean())/X.std()
# print(X.mean()[22], X.std()[22])
df_scaled.update(X)
df_scaled
#features
features_pre = df_scaled.iloc[:,:-1]
#features
features_pre = df_scaled.iloc[:,:-1]

print('# of features before drop:', features_pre.shape[1])

#create correlation matrix
corr_matrix_before = features_pre.corr().abs()

#set a cutoff threshold (usually 0.9 or 0.85)
cut_th = 0.9

# of features before drop: 662


In [121]:
# features_pre

In [7]:
# np.save("model-10-final-training-scaled-data.npy",features_pre.to_numpy())

# Machine learning

In [8]:
#import scikit learn packages
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import (recall_score,accuracy_score,confusion_matrix, f1_score, precision_score, 
                             auc,roc_auc_score,roc_curve, precision_recall_curve,classification_report)

In [124]:
#### Model-10 ####

from sklearn.neural_network import MLPClassifier

#keeping the upper (right) triangular matrix without the main diagonal
upper = corr_matrix_before.where(np.triu(np.ones(corr_matrix_before.shape), k=1).astype(np.bool))

#removed a column (i.e., feature), if any element(s) in the column of the upper triangular matrix is/are 
#larger than the threshold value
cut_th = 0.95
to_drop = [column for column in upper.columns if (any(upper[column] > cut_th) and column not in [6271, 6276, 6758, 6762])]
print("dropped columns:", len(to_drop))
df_dropped = df_scaled.drop(columns = to_drop)


X = df_dropped.iloc[:,:-1]
y = df_dropped['state']

if type(X) is not np.ndarray:
    X, y = X.to_numpy(),y.to_numpy()
else:
    print('X,y already converted to ndarray')

print(X.shape, y.shape)
# # print(y.shape)
#convert labels to one hot code for MLP
one_hot_y = y[:,None]==np.array([1,2])
# X_train, X_test, y_train, y_test = train_test_split(X,one_hot_y,test_size = 0.20, random_state =4) 
X1 = X[:15000,]
X2 = X[15000:,]
y1 = one_hot_y[:15000,]  ## for MLP
y2 = one_hot_y[15000:,] ## for MLP

# print(X1.shape, X2.shape, y1.shape, y2.shape)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size = 0.20, random_state =1) 
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,test_size = 0.20, random_state =1) 

X_train = np.concatenate((X1_train,X2_train),axis=0)
y_train = np.concatenate((y1_train,y2_train),axis=0)
X_test = np.concatenate((X1_test,X2_test),axis=0)
y_test = np.concatenate((y1_test,y2_test),axis=0)
print(X_train)


/var/folders/6n/3nqlb7m93wg2vxy4dg00ggzh0000gn/T/ipykernel_47964/255855387.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix_before.where(np.triu(np.ones(corr_matrix_before.shape), k=1).astype(np.bool))


dropped columns: 42
(30000, 620) (30000,)
[[ 1.90008464  1.97686647  0.67368321 ...  0.28215796  1.55231471
  -0.3265295 ]
 [-0.33840144  0.0619965  -0.84642334 ...  0.93215241  0.73220886
  -0.10403966]
 [-0.34974641  0.12145749 -0.52864777 ... -0.19206827  1.06780244
  -0.61708815]
 ...
 [-1.24389149  1.75802147  0.80838026 ... -1.65977353  0.96473615
  -0.99285142]
 [ 0.44826066 -1.36922101 -1.09743236 ...  1.68634421 -1.10581234
  -1.02371726]
 [-0.41218783 -0.58829468 -1.08572197 ...  0.0797455  -1.02694162
  -0.43441097]]


In [126]:
# one_hot_y = y[:,None]==np.array([1,2])
# print(one_hot_y)

In [51]:
np.save("model10-after_dropped-training-scaled-data.npy",df_dropped.to_numpy())

In [30]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(24000, 91) (24000, 2) (6000, 91) (6000, 2)


## Extract features selected to train the model

In [52]:
## Will use this to drop columns in blind datasets
keep = [column for column in df_dropped.iloc[:,:-1].columns]
print(upper.shape)
print("keep columns:", len(keep))
print(keep)


(121, 121)
keep columns: 91
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 39, 41, 42, 43, 44, 45, 47, 48, 50, 52, 53, 54, 55, 56, 58, 59, 63, 64, 66, 67, 68, 69, 70, 72, 74, 75, 77, 78, 80, 81, 83, 85, 88, 89, 92, 96, 97, 99, 100, 101, 102, 103, 105, 107, 108, 109, 110, 111, 113, 114, 116, 118, 119, 120]


## Hyperparameter tuning

In [32]:
# # clf=MLP.fit(X_train, y_train)
### For MLP Classifier
mlp_gs = MLPClassifier(max_iter=500)
parameter_space = {
    'hidden_layer_sizes': [(5,),(10),(20,)], ## (root-over (input layer size * output layer size))
#     'hidden_layer_sizes': [(30), (50,),(70,)], ## (root-over (input layer size * output layer size))
#     'hidden_layer_sizes': [(20,)],
#     'activation': ['relu', 'tanh'],
    'activation': ['relu'],
# #     'activation': ['tanh', 'relu'],
# #     'activation': ['logistic','tanh'],
# #     'solver': ['sgd', 'adam'],
    'solver': ['adam'],
# #     'learning_rate': ['constant','adaptive'],
    'learning_rate': ['adaptive'],
    'verbose': ['True'],
}
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train) # X is train samples and y is the corresponding labels


Iteration 1, loss = 0.35953303
Iteration 2, loss = 0.04526322
Iteration 3, loss = 0.01728222
Iteration 4, loss = 0.00917314
Iteration 5, loss = 0.00571747
Iteration 6, loss = 0.00392574
Iteration 7, loss = 0.00287536
Iteration 8, loss = 0.00219750
Iteration 9, loss = 0.00173726
Iteration 10, loss = 0.00140359
Iteration 11, loss = 0.00115874
Iteration 12, loss = 0.00096873
Iteration 13, loss = 0.00082394
Iteration 14, loss = 0.00070750
Iteration 15, loss = 0.00061229
Iteration 16, loss = 0.00053415
Iteration 17, loss = 0.00047040
Iteration 18, loss = 0.00041678
Iteration 19, loss = 0.00037056
Iteration 20, loss = 0.00033241
Iteration 21, loss = 0.00029828
Iteration 22, loss = 0.00026920
Iteration 23, loss = 0.00024405
Iteration 24, loss = 0.00022195
Iteration 25, loss = 0.00020255
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


GridSearchCV(cv=3, estimator=MLPClassifier(max_iter=500), n_jobs=-1,
             param_grid={'activation': ['relu'],
                         'hidden_layer_sizes': [(5,), 10, (20,)],
                         'learning_rate': ['adaptive'], 'solver': ['adam'],
                         'verbose': ['True']})

In [33]:
print(clf.score(X_test, y_test))
print('Best parameters found:\n', clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


1.0
Best parameters found:
 {'activation': 'relu', 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'adam', 'verbose': 'True'}
1.000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': (5,), 'learning_rate': 'adaptive', 'solver': 'adam', 'verbose': 'True'}
1.000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': 10, 'learning_rate': 'adaptive', 'solver': 'adam', 'verbose': 'True'}
1.000 (+/-0.000) for {'activation': 'relu', 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'adam', 'verbose': 'True'}


In [22]:
# import joblib
 
# # # Save the model as a pickle in a file
# joblib.dump(clf, 'cas13_trainedModel9_HEPN1-2_MLP.pkl') #''activation': 'relu', 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'adam', 'verbose': 'True'

## Train the model and save

In [34]:
#MLP classifier parameters, hyperparameters were tuned for an accuracy of 1
MLP = MLPClassifier(hidden_layer_sizes = (20), activation = 'relu', solver = 'adam', max_iter= 500) ##Model=10

In [35]:
clf=MLP.fit(X_train, y_train)

In [36]:
#Predicting y for X_val
y_true, y_pred = y_test , clf.predict(X_test)

In [37]:
# print(y_pred)

In [38]:
def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

#Importing Confusion Matrix
from sklearn.metrics import confusion_matrix
#Comparing the predictions against the actual observations in y_val
cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))

#Printing the accuracy
print("Accuracy of MLPClassifier :", accuracy(cm))

Accuracy of MLPClassifier : 1.0


In [39]:
clf.score(X_test, y_test)

1.0

In [40]:
import joblib
 
# # # Save the model as a pickle in a file
joblib.dump(clf, 'trainedModel10_MLP.pkl') #''activation': 'relu', 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'adam', 'verbose': 'True'

['trainedModel10_MLP.pkl']

## Load new data to see whether the model can predict the class

In [43]:
import joblib
# Load the model from the file
clf_from_joblib = joblib.load('trainedModel10_MLP.pkl')


In [91]:
## For normalization of the blind dataset
import numpy as np
X1 = np.load("model10-after_dropped-training-scaled-data.npy", allow_pickle=True)
print(X1.shape)
# # print(np.arange(X.shape[1]))
X = pd.DataFrame(X1[:,:-1])
print(X)

(30000, 92)
             0         1         2         3         4         5         6   \
0     -0.629672 -0.323976 -0.346787   0.29724  0.425553  -0.46048 -0.819467   
1     -0.874482 -1.059826 -0.915018 -0.340862 -0.113874 -0.913905 -1.287489   
2     -0.908217 -1.097013 -1.074829 -0.443913 -0.224113 -1.180518  -1.73408   
3      -0.90639 -0.934969 -0.844596 -0.222192 -0.125362 -1.064765 -1.247103   
4     -0.886505 -0.430737 -0.820862 -0.457413 -0.976199 -1.223934  -1.42749   
...         ...       ...       ...       ...       ...       ...       ...   
29995  0.573253  1.109363  1.052025 -0.833726  0.926996  1.534224  1.702952   
29996  0.195631  1.009605  1.404647 -0.813512  0.829126  1.441796  1.108841   
29997   0.07855  0.667228  0.385573 -0.569547  0.629942  0.680768  0.228927   
29998  0.048143  0.519618  0.571561 -0.793605  0.606106  1.462611  0.684186   
29999 -0.023088  0.456379 -0.148084 -1.162912  0.514142  0.377932 -0.009596   

             7         8         9   ..

In [96]:
a = np.load("../load_new_system_dist_feature.npy")
a_temp = 1/a

print(a_temp.shape)
print(a_temp) # inverse distance



(1500, 121)
[[0.04645908 0.04931578 0.05198285 ... 0.05055775 0.04990144 0.04021748]
 [0.04599679 0.04929151 0.04844708 ... 0.0528009  0.05077157 0.04151761]
 [0.04573089 0.04904905 0.04831232 ... 0.05321961 0.0487054  0.03986515]
 ...
 [0.04478709 0.0485464  0.04690447 ... 0.04253328 0.03917655 0.03396482]
 [0.04312899 0.04682549 0.04555294 ... 0.04179424 0.0397465  0.0334677 ]
 [0.04268947 0.04283015 0.04445565 ... 0.04000773 0.03901433 0.03299315]]


In [97]:
drop_ca = [i for i in np.arange(tgRNA.shape[1]) if i not in keep]
print(len(keep))
print(len(drop_ca))

91
30


In [102]:

df3=pd.DataFrame(a_temp)

# df4 = df3.drop(df3.columns[drop_ca],axis = 1)
df4 = df3.drop(columns = drop_ca)
x1 = df4.to_numpy()
print(x1.shape)
# print(df4)

# #scaled/normalized by each feature

for i in (np.arange(X.shape[1])):
    x1[:,i] = (x1[:,i] - X[i].mean(axis=0))/X[i].std(axis=0)
print(x1[0,0])
print(x1)
# df1_scaled= (df1_scaled-X2.mean())/X2.std()

(1500, 91)
0.04645907944682835
[[0.04645908 0.04931578 0.05198285 ... 0.05055775 0.04990144 0.04021748]
 [0.04599679 0.04929151 0.04844708 ... 0.0528009  0.05077157 0.04151761]
 [0.04573089 0.04904905 0.04831232 ... 0.05321961 0.0487054  0.03986515]
 ...
 [0.04478709 0.0485464  0.04690447 ... 0.04253328 0.03917655 0.03396482]
 [0.04312899 0.04682549 0.04555294 ... 0.04179424 0.0397465  0.0334677 ]
 [0.04268947 0.04283015 0.04445565 ... 0.04000773 0.03901433 0.03299315]]


In [104]:
# Use the loaded model to make predictions
pred = clf_from_joblib.predict(x1)

In [111]:

print(pred.shape)
c = 0
d = 0
e = 0
f = []
for i, j in enumerate(pred.argmax(axis=1)):
    if ((pred[i,0] == 1) & (pred[i,1] == 0)):
        c += 1
        f.append(i)
    if ((pred[i,0] == 0) & (pred[i,1] == 1)):
        d += 1
    if ((pred[i,0] == 0) & (pred[i,1] == 0)):
        e += 1
    if ((pred[i,0] == 1) & (pred[i,1] == 1)):
        e += 1
    
print(c,d,e)
print("% of tgRNA-like state:", d/pred.shape[0])
print("% of crRNA-like state:", c/pred.shape[0])
print("% of undecided states:", e/pred.shape[0])
# print(len(f))
# print(f)

(1500, 2)
0 1500 0
% of active-like state: 1.0
% of inactive-like state: 0.0
% of undecided states: 0.0


In [115]:
# print(atgRNA[80:85,:])
# print(clf_from_joblib.predict(X1[4:50,:]))
prob_results = clf_from_joblib.predict_proba(x1[100:110])
print(prob_results)

[[0.20841629 0.69930449]
 [0.21985708 0.69776183]
 [0.22248637 0.69862823]
 [0.23666826 0.68403403]
 [0.23330607 0.69032897]
 [0.23542974 0.68441936]
 [0.22054212 0.70115378]
 [0.23405192 0.68905447]
 [0.22453735 0.69959222]
 [0.22387368 0.69549661]]
